## Clasificación de género con imágenes de manos — Guided Filter

## Este notebook implementa un modelo basado en redes neuronales convolucionales (CNN), utilizando un preprocesamiento con filtro guiado para suavizar las imágenes y realzar las características globales de la mano.

## 1. Importación de librerías necesarias

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers.legacy import SGD
print("TensorFlow version:", tf.__version__)
print("OpenCV version:", cv2.__version__)
print("NumPy version:", np.__version__)

## 2. Parámetros de configuración

In [ ]:
IMG_SIZE    = (224, 224)
BATCH_SIZE  = 32
RADIUS      = 10    # radio del filtro guiado
EPS         = 0.01   # parámetro epsilon de suavizado

csv_path   = "../../dataset/HandInfo.csv"
image_root = "../../dataset/image"

## 3. Carga del CSV y codificación de etiquetas de género

In [ ]:
df = pd.read_csv(csv_path)
df["label"] = df["gender"].map({"male": 0, "female": 1})
NUM_CLASSES = df["label"].nunique()

## 4. Normalización Z-score para estandarizar las imágenes

In [ ]:
def z_score_normalization(image: np.ndarray) -> np.ndarray:
    mean = image.mean()
    std  = image.std()
    return (image - mean) / (std + 1e-7)

## 5. Preprocesamiento de imágenes con filtro guiado

In [ ]:
def preprocess_image_guided(path_bytes: bytes) -> np.ndarray:
    path = path_bytes.decode("utf-8")
    img  = cv2.imread(path).astype(np.float32)
    if img is None:
        return np.zeros((*IMG_SIZE, 3), dtype=np.float32)
    img = cv2.resize(img, IMG_SIZE)
    gf  = cv2.ximgproc.createGuidedFilter(guide=img, radius=RADIUS, eps=EPS)
    low = gf.filter(img)
    return z_score_normalization(low)

def tf_preprocess(path, label):
    out = tf.numpy_function(preprocess_image_guided, [path], tf.float32)
    out.set_shape((*IMG_SIZE, 3))
    return out, label

## 6. Creación de conjuntos de datos (train, val, test)

In [ ]:
def create_dataset(split: str):
    folder = os.path.join(image_root, split)
    files  = set(os.listdir(folder))
    subdf  = df[df["imageName"].isin(files)].sort_values("imageName")
    paths  = [os.path.join(folder, fn) for fn in subdf["imageName"]]
    labels = subdf["label"].values
    ds     = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds     = ds.map(tf_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    if split == "train":
        ds = ds.shuffle(1000)
    return ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

train_ds = create_dataset("train")
val_ds   = create_dataset("val")
test_ds  = create_dataset("test")
print("Train dataset size:", len(train_ds))
print("Validation dataset size:", len(val_ds))
print("Test dataset size:", len(test_ds))

## 7. Definición de bloques residuales y arquitectura tipo ResNet-light

In [ ]:
def residual_block(x, filters, downsample=False):
    identity = x
    stride   = 2 if downsample else 1

    x = layers.Conv2D(filters, 3, strides=stride, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters, 3, strides=1, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    if downsample or identity.shape[-1] != filters:
        identity = layers.Conv2D(filters, 1, strides=stride, use_bias=False)(identity)
        identity = layers.BatchNormalization()(identity)

    x = layers.Add()([x, identity])
    return layers.ReLU()(x)

def build_resnet(input_shape=(224,224,3), num_classes=2):
    inp = layers.Input(shape=input_shape)
    x   = layers.Conv2D(64, 7, strides=2, padding="same", use_bias=False)(inp)
    x   = layers.BatchNormalization()(x)
    x   = layers.ReLU()(x)
    x   = layers.MaxPooling2D(3, strides=2, padding="same")(x)
    for _ in range(2):
        x = residual_block(x, 64)
    for i in range(3):
        x = residual_block(x, 128, downsample=(i==0))
    for i in range(4):
        x = residual_block(x, 256, downsample=(i==0))
    x   = layers.GlobalAveragePooling2D()(x)
    out = layers.Dense(num_classes, activation="softmax")(x)
    return models.Model(inp, out)

## 8. Compilación del modelo

In [ ]:
model = build_resnet(num_classes=NUM_CLASSES)
model.compile(
    optimizer=SGD(learning_rate=0.09, momentum=0.9, decay=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

## 9. Callback personalizado para evaluar sobre el conjunto de test

In [ ]:
class TestCallback(tf.keras.callbacks.Callback):
    def __init__(self, test_ds):
        super().__init__()
        self.test_ds = test_ds
        self.test_losses, self.test_accuracies = [], []
    def on_epoch_end(self, epoch, logs=None):
        loss, acc = self.model.evaluate(self.test_ds, verbose=0)
        self.test_losses.append(loss)
        self.test_accuracies.append(acc)
        print(f"  → Test loss: {loss:.4f} — Test acc: {acc:.4f}")

test_cb = TestCallback(test_ds)

## 10. Scheduler de reducción de Learning Rate

In [ ]:
def lr_schedule(epoch, lr):
    if epoch == 3:
        return lr * 0.7
    return lr

lr_cb = tf.keras.callbacks.LearningRateScheduler(lr_schedule, verbose=1)

## 11. Entrenamiento del modelo

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    callbacks=[lr_cb, test_cb],
    verbose=1
)

## 12. Guardado del modelo entrenado

In [ ]:
model.save("guided_model.h5")

## 13. Gráficas de precisión y pérdida: Train vs Test

In [ ]:
epochs = range(1, len(history.history["accuracy"]) + 1)
train_acc = history.history["accuracy"]
test_acc  = test_cb.test_accuracies
train_loss= history.history["loss"]
test_loss = test_cb.test_losses

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,4))

# Accuracy
ax1.plot(epochs, train_acc, marker='o', color='blue', label='Train')
ax1.plot(epochs, test_acc,  marker='s', color='red',  label='Test')
ax1.set_title("Accuracy over epochs")
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Accuracy")
ax1.set_xticks(epochs)
ax1.legend()
ax1.grid(True)

# Loss
ax2.plot(epochs, train_loss, marker='o', color='blue', label='Train')
ax2.plot(epochs, test_loss,  marker='s', color='red',  label='Test')
ax2.set_title("Loss over epochs")
ax2.set_xlabel("Epoch")
ax2.set_ylabel("Loss")
ax2.set_xticks(epochs)
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()